In [ ]:
from gensim.models import KeyedVectors
 
model_dir = 'data/entity_vector/entity_vector.model.bin'
model = KeyedVectors.load_word2vec_format(model_dir, binary=True)
results = model.most_similar(u'[ディープラーニング]')
for result in results:
    print(result)

In [6]:
results = model.most_similar('[ディープラーニング]')
for result in results:
    print(result)

('[機械学習]', 0.6342067122459412)
('[グリッド・コンピューティング]', 0.5937871932983398)
('[タンパク質構造予測]', 0.5929729342460632)
('[クリティカルパス法]', 0.5910710692405701)
('[デジタル信号処理]', 0.5864137411117554)
('[量子コンピュータ]', 0.5849090814590454)
('[分散コンピューティング]', 0.5813807249069214)
('[分子動力学]', 0.577800989151001)
('[データマイニング]', 0.5750172138214111)
('[モンテカルロ法]', 0.5733920335769653)


In [7]:
results = model.most_similar('旅行')
for result in results:
    print(result)

('新婚旅行', 0.7033272981643677)
('[旅行]', 0.6934359669685364)
('外遊', 0.6871017217636108)
('海外旅行', 0.6735522150993347)
('家族旅行', 0.6658696532249451)
('一人旅', 0.6619066596031189)
('[新婚旅行]', 0.654478132724762)
('旅', 0.6521649360656738)
('[グランドツアー]', 0.6485979557037354)
('船旅', 0.6366738080978394)


In [8]:
results = model.most_similar('[旅行]')
for result in results:
    print(result)

('[海外旅行]', 0.764588475227356)
('個人旅行', 0.7003886103630066)
('海外旅行', 0.699029803276062)
('旅行', 0.6934360265731812)
('国内旅行', 0.6846235990524292)
('短期滞在', 0.6807036995887756)
('[旅]', 0.678523063659668)
('[バックパッカー]', 0.6708515286445618)
('[パッケージツアー]', 0.6653098464012146)
('鉄道旅行', 0.6575880646705627)


In [66]:
from scipy.spatial import distance
import MeCab

def tokenize(s):
    mecab = MeCab.Tagger("-Owakati")
    return mecab.parse(s).split()

def get_vector(token):
    try:
        return model.get_vector(f'[{token}]')
    except KeyError:
        try:
            return model.get_vector(token)
        except KeyError:
            return np.zeros(200)

def calc_similarity(s1, s2):
    tokens1 = tokenize(s1)
    tokens2 = tokenize(s2)
    vec1 = np.mean([get_vector(t) for t in tokens1], axis=0)
    vec2 = np.mean([get_vector(t) for t in tokens2], axis=0)
    return 1 - distance.cosine(vec1, vec2)

In [67]:
calc_similarity('コンサートとかには行きますか？', 'コンサートは行かないです')

0.6737689971923828

In [68]:
calc_similarity('コンサートとかには行きますか？', '織田信長の野望')

0.27168598771095276

In [69]:
calc_similarity('豊臣秀吉の狙いはなんだったのか？', '織田信長の野望')

0.5223014950752258

In [92]:
def topic_generator(text_gen, threshold=0.3, max_utterances=50, min_utterances=5, delay=3):
    cur_topic = [next(text_gen)]
    
    for text in text_gen:
        if (
            calc_similarity('\n'.join(cur_topic[:-delay]), '\n'.join(cur_topic[-delay:] + [text])) < threshold
            or len(cur_topic) >= max_utterances
        ) and len(cur_topic) >= min_utterances:
            yield cur_topic[:-delay]
            cur_topic = cur_topic[-delay:] + [text]
        else:
            cur_topic.append(text)

In [93]:
import jsonlines
import unicodedata

def transcript_generator(path):
    with jsonlines.open(path) as reader:
        for data in reader:
            yield preprocess_transcript(data['transcript'])

def preprocess_transcript(transcript):
    transcript = unicodedata.normalize('NFKC', transcript)
    transcript = transcript.replace('|', ' ')

    return transcript

In [131]:
topic_gen = topic_generator(
    transcript_generator('data/golden_transcripts/1911F2002.jsonl'),
    threshold=0.8,
    max_utterances=100,
    min_utterances=5,
    delay=2,
)

for topic in topic_gen:
    print(len(topic), topic)
    

3 [' これから本番の収録を開始します。', ' はじめまして・・私の名前は「メイ」です。「対話の実験」の手伝いをしています。今日はよろしくお願いします。', 'よろしくお願いします']
14 [' きょうはここまでなにで来られましたか?乗り物は乗りましたか?', 'そうですね 電車で 来て歩いて来ました', ' ここに来るまで迷いませんでしたか・・', 'ここに来るまでは人に教えてもらったので 迷いませんでした', ' 大学の中でも、たてものが奥にあるので、みなさん来るのが「たいへん」でしたとよく言われます・・', '(F あー)そうなんですか 確かに奥にありました', ' きょうは来ていただいてありがとうございます!', ' それでは「食べ物」について話しましょう!', '(F はい)', ' 甘いものはお好きですか?', '甘いものは洋菓子じゃなくて和菓子のほうが好きですね', ' 普段はどこであまいものを買いますか?', '普段はいやあコンビニが多いですね', ' 和菓子はどんなものですか']
47 ['和菓子どんなもの 大福とかみたらし団子とか', ' 大福、ですか?', 'はい', ' コンビニですか。よく、行くのですか。', 'コンビニよく行きますね', ' そうなんですね、ちょっと特別なおかしを買うときは近くのお店とか「デパートの地下」とか、利用しますか?', '(F あー)利用しますね', ' デパートのホームページを見て、キラキラしていて楽しそうと思うのですが、実際はいかがですか?', '実際は人が多くてしんどい所ですね', ' そうなんですか.', ' その中でもおすすめのスイーツはありますか?', '(F えー) は なんかケーキ (D わか)', ' そうなんですか.', 'うん', ' 私の友人はあんこ系が好きだそうです。あずきときなこは、体や肌にいいそうで、よく食べてるそうです・・', '(F ふーん) (F へー)', ' おばあちゃんがよく家で出してくれていたと言っていました。', '(F えー)いいおばあちゃんですね', ' いい思い出だったようです。', '(F はあ)', ' からい食べものとはどんな食べものが好きですか?', '辛い食べ物は (F うーん) なんか麺も好きやしご飯丼ぶり系も好きです', ' 塩辛いほうですか

In [127]:
a = [' これから本番の収録を開始します。', ' はじめまして・・私の名前は「メイ」です。「対話の実験」の手伝いをしています。今日はよろしくお願いします。', 'よろしくお願いします', ' きょうはここまでなにで来られましたか?乗り物は乗りましたか?', 'そうですね 電車で 来て歩いて来ました', ' ここに来るまで迷いませんでしたか・・', 'ここに来るまでは人に教えてもらったので 迷いませんでした', ' 大学の中でも、たてものが奥にあるので、みなさん来るのが「たいへん」でしたとよく言われます・・', '(F あー)そうなんですか 確かに奥にありました', ' きょうは来ていただいてありがとうございます!', ' それでは「食べ物」について話しましょう!', '(F はい)', ' 甘いものはお好きですか?', '甘いものは洋菓子じゃなくて和菓子のほうが好きですね', ' 普段はどこであまいものを買いますか?', '普段はいやあコンビニが多いですね', ' 和菓子はどんなものですか']
b = ['和菓子どんなもの 大福とかみたらし団子とか', ' 大福、ですか?', 'はい', ' コンビニですか。よく、行くのですか。', 'コンビニよく行きますね', ' そうなんですね、ちょっと特別なおかしを買うときは近くのお店とか「デパートの地下」とか、利用しますか?', '(F あー)利用しますね', ' デパートのホームページを見て、キラキラしていて楽しそうと思うのですが、実際はいかがですか?', '実際は人が多くてしんどい所ですね', ' そうなんですか.', ' その中でもおすすめのスイーツはありますか?', '(F えー) は なんかケーキ (D わか)', ' そうなんですか.', 'うん', ' 私の友人はあんこ系が好きだそうです。あずきときなこは、体や肌にいいそうで、よく食べてるそうです・・', '(F ふーん) (F へー)', ' おばあちゃんがよく家で出してくれていたと言っていました。', '(F えー)いいおばあちゃんですね', ' いい思い出だったようです。', '(F はあ)', ' からい食べものとはどんな食べものが好きですか?', '辛い食べ物は (F うーん) なんか麺も好きやしご飯丼ぶり系も好きです', ' 塩辛いほうですか、トウガラシのからさとかですか?', '(D し) 塩辛いほうですね', ' 具体的な食べ物の名前を教えてください。', '具体的な (F えー) オムライスと 牛丼と なんかそんな感じの', ' 牛丼、ですか?', 'はい', ' それは「最近」食べられたんですか?', '最近は食べてないです', ' そうなんですね、お店ですか、ご自宅ですか?', 'お店ですね', ' なるほど、そうなんですね、', '(F はい)', ' オムライスそれは「最近」食べられたんですか?', 'オムライスは食べました', ' オムライス、どこで召し上がったのですか・・', 'オムライスは家で食べました', ' そうですか、それは美味しそうですね・・', ' お友達とランチに行ったりしますか?', 'よく行きますね', ' なるほど、どんなところに行きますか?和食ですか洋食ですか?それともはやりのカフェとかですか?', '(D よう) 流行りのカフェですね', ' 例えば?', '例えば あれどこやろ 梅田にある ような 感じ', ' うめだ、ですか?', ' 、ですか?']
calc_similarity('\n'.join(a[:2]), '\n'.join(b[:3]))

0.6496319770812988

In [49]:
list(gen)

[2, 3]

In [83]:
[1,2,3,4][:-2]

[1, 2]